# Differences in MT and SP in parasha #14: Va’era (Exodus 6:2-9:35)

## Table of Content <a class="anchor" id="TOC"></a> (ToC)

* <a href="#bullet1">1 - Introduction</a>
* <a href="#bullet2">2 - Load Text-Fabric app and data</a>
* <a href="#bullet3">3 - Compare surface texts of SP and MT</a>
* <a href="#bullet4">4 - Compare texts using minimum Levenshtein distance</a>
* <a href="#bullet5">5 - Comparison of spelling of proper nouns between SP and MT</a>
* <a href="#bullet6">6 - References and acknowledgement</a>
* <a href="#bullet7">7 - Required libraries</a>
* <a href="#bullet8">8 - Notebook version details</a>

# 1 - Introduction <a class="anchor" id="bullet1"></a>
##### [Back to ToC](#TOC)

The Samaritan Pentateuch (SP) is a version of the Torah preserved by the Samaritan community, differing from the Masoretic Text (MT) in several aspects, including language, orthography, and occasionally theological emphasis. This notebook compares the text of the Masoretic Text, based on the BHSA dataset in Text-Fabric, with the Samaritan Pentateuch, also available as a Text-Fabric dataset.<a href="#ref1"><sup>1</sup></a>

In this analysis, we focus on comparing the text of the verses in a specific parasha, highlighting differences in wording and orthography. Additionally, special attention is given to spelling variations of proper nouns between the two traditions. This notebook draws inspiration from the notebook provided by Martijn Naaijer<a href="#ref2"><sup>2</sup></a> and aims to explore the textual nuances between these two important versions of the Torah.

# 2 - Load Text-Fabric app and data <a class="anchor" id="bullet2"></a>
##### [Back to ToC](#TOC)

The following code will load the Text-Fabric version of the [Samaritan Pentatuch](https://github.com/DT-UCPH/sp), the [Biblia Hebraica Stuttgartensia (Amstelodamensis)](https://etcbc.github.io/bhsa/) together with the additonal parasha related features from [tonyjurg/BHSaddons](https://github.com/tonyjurg/BHSaddons).

In [1]:
from tf.app import use

# Load the SP data, and rename the node features class F,
# the locality class L and the text class T, 
# then they cannot be overwritten while loading the MT.
SP = use('DT-UCPH/sp', version='3.4')
Fsp, Lsp, Tsp = SP.api.F, SP.api.L, SP.api.T

# Do the same for the MT dataset (BHSA) together with BHSaddons 
MT = use('etcbc/bhsa', version='2021',mod="tonyjurg/BHSaddons/tf/")
Fmt, Lmt, Tmt = MT.api.F, MT.api.L, MT.api.T

**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,5,79878.40,100
chapter,187,2135.79,100
verse,5841,68.38,100
word,114890,3.48,100
sign,399392,1.00,100


**Locating corpus resources ...**

Name,# of nodes,# slots / node,% coverage
book,39,10938.21,100
chapter,929,459.19,100
lex,9230,46.22,100
verse,23213,18.38,100
half_verse,45179,9.44,100
sentence,63717,6.70,100
sentence_atom,64514,6.61,100
clause,88131,4.84,100
clause_atom,90704,4.70,100
phrase,253203,1.68,100


# 3 - Compare surface texts of SP and MT <a class="anchor" id="bullet3"></a>
##### [Back to ToC](#TOC)

In this section, we compare the surface texts of the Samaritan Pentateuch (SP) and the Masoretic Text (MT) at the verse level. By analyzing the wording and structure of these texts, we aim to identify variations.

In [2]:
# find all word nodes for this parasha (we can either use the transliterated name or the sequence number)
parashaQuery = '''
verse parashanum=14
'''
parashaResults = MT.search(parashaQuery)

  0.01s 121 results


In [3]:
# Extract book, chapter, and verse information
bookChapterVerseList = [
    Tmt.sectionFromNode(verse[0]) for verse in parashaResults
]

# Store parashname, start and end verse for future use
startNode=parashaResults[0][0]
endNode=parashaResults[-1][0]
parashaNameHebrew=Fmt.parashahebr.v(startNode)
parashaNameEnglish=Fmt.parashatrans.v(startNode)
bookStart,chapterStart,startVerse=Tmt.sectionFromNode(startNode)
parashaStart=f'{bookStart} {chapterStart}:{startVerse}'
bookEnd,chapterEnd,startEnd=Tmt.sectionFromNode(endNode)
parashaEnd=f'{chapterEnd}:{startEnd}'
htmlStart='<html><body>'
htmlFooter=f'<p>Data generated by `delta_mt_and_sp.ipynb` at `<a href=\"https://github.com/tonyjurg/Parashot\" target=\"_blank\">github.com/tonyjurg/Parashot</a>`</p></body></html>`'

In [4]:
# Function to reconstruct verses
def reconstructVerses(F, L, T, textFeature, inputList):
    """Reconstruct text for each verse."""
    verseTexts = {}
    for verseName in inputList:
        verseText = ''
        verseNode = T.nodeFromSection(verseName)
        wordNodes = L.d(verseNode, 'word')
        for wordNode in wordNodes:
            wordText = eval(f'F.{textFeature}.v(wordNode)')
            trailer = F.trailer.v(wordNode)
            if wordText:
                verseText += wordText + (trailer if trailer else ' ')
        verseTexts[verseName] = verseText.strip()
    return verseTexts
    
SPverses = reconstructVerses(Fsp, Lsp, Tsp, 'g_cons', bookChapterVerseList)
MTverses = reconstructVerses(Fmt, Lmt, Tmt, 'g_cons', bookChapterVerseList)

In [5]:
from difflib import SequenceMatcher
from IPython.display import HTML, display

def highlightMatches(baseText, comparisonText):
    matcher = SequenceMatcher(None, baseText, comparisonText)
    highlightedComparisonText = "" 
    for tag, i1, i2, j1, j2 in matcher.get_opcodes():
        if tag == "equal":  # Identical parts
            highlightedComparisonText += comparisonText[j1:j2]
        else:  # Non-matching parts
            highlightedComparisonText += f'<mark>{comparisonText[j1:j2]}</mark>'  
    return highlightedComparisonText

def cleanText(text):
    replacements = [
         # for the transcoded strings
         ('00_P', ''),  # Remove '00_P'
         ('00_S', ''),  # Remove '00_S'
         ('00', ''),    # Remove '00'
         ('&', ' '),    # Replace '&' with a space
         # for the Hebrew strings
         ('ס ', ''),    # Final Samekh
         ('פ ', ''),    # Final Pe
         ('׃', ''),     # End of verse
         ('־',' ')      # maqaf
    ]
    # Apply each replacement
    for old, new in replacements:
        text = text.replace(old, new)
    return text

# Function to format and highlight verse differences between MT and SP
def formatAndHighlight(label, MTverseText, SPverseText):
    book, chapter, verse = label
    MTverseNode = Tmt.nodeFromSection(label)
    MTtext = cleanText(Tmt.text(MTverseNode, "text-orig-plain"))
    SPverseNode = Tsp.nodeFromSection(label)
    SPtext = Tsp.text(SPverseNode)
    SPmarkedText = highlightMatches(MTtext, SPtext)
    MTmarkedText = highlightMatches(SPtext, MTtext)
    formattedDiff = (
        f'<h4><a href=\"https://www.stepbible.org/?q=version=NASB2020&reference='
        f'{book}.{chapter}:{verse}&options=HNVUG\" target=\"_blank\">{book} {chapter}:{verse}</a></h4>'
        f'<p><b>SP:</b> {SPmarkedText}<br><b>MT:</b> {MTmarkedText}</p>'
    )
    return formattedDiff

# Gather differences into an HTML string
htmlContent = f'<h2>Differences between MT and SP for parasha {parashaNameEnglish} ({parashaStart}-{parashaEnd})</h2>'
for label, MTverseText in MTverses.items():
    SPverseText = SPverses.get(label, '')
    MTverseText = cleanText(MTverseText)
    if MTverseText != SPverseText:  # Check for differences
        difference = formatAndHighlight(label, MTverseText, SPverseText)
        htmlContent += difference

# Save the content to an HTML file
fileName = f"differences_MT_SP({parashaNameEnglish.replace(' ','%20')}).html"
with open(fileName, "w", encoding="utf-8") as file:
    file.write(htmlContent)

# Display the content in the notebook
display(HTML(htmlContent))

# wrap html header and footer and display a download button
htmlContentFull = f'{htmlStart}{htmlContent}{htmlFooter}'
downloadButton = f"""
<a download="{fileName}" href="data:text/html;charset=utf-8,{htmlContentFull.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;').replace('"', '&quot;').replace("'", '&#39;')}" target="_blank">
    <button>Download Differences as HTML</button>
</a>
"""
display(HTML(downloadButton))

# 4 - Compare texts using minimum Levenshtein distance<a class="anchor" id="bullet4"></a>
##### [Back to ToC](#TOC)

The Levenshtein distance measures the minimum number of single-character edits (insertions, deletions, or substitutions) needed to transform one text into another, providing a quantitative way to compare textual differences. For comparing the Masoretic Text and Samaritan Pentateuch, it highlights variations in spelling, word order, or minor textual changes. 
In the context of the Levenshtein distance (in the script below `threshold`), a higher number indicates greater dissimilarity between two texts, meaning more edits (insertions, deletions, or substitutions) are needed to transform one text into the other.

In [6]:
from Levenshtein import distance
from IPython.display import HTML, display

threshold = 15

# Create an HTML string to store the output
htmlContent = f'<h2>Levenshtein distance >{threshold} between MT and SP for parasha {parashaNameEnglish} ({parashaStart}-{parashaEnd})</h2>'

# Create header
MT.dm(f'### Levenshtein distance >{threshold} between MT and SP for parasha {parashaNameEnglish} ({parashaStart}-{parashaEnd})')

# Generate the HTML content
for label, MTverseText in MTverses.items():
    SPverseText = SPverses.get(label, '')
    levDistance = distance(MTverseText, SPverseText)  # Calculate the distance
    if levDistance > threshold:
        formattedDiff = formatAndHighlight(label, MTverseText, SPverseText)
        formattedDiff += f'<p>Levenshtein Distance: {levDistance}</p>'  # Add the distance
        MT.dm(formattedDiff)
        htmlContent += formattedDiff  # Append to the HTML content

# Save the content to an HTML file
fileName = f"levenshtein_differences_MT_SP({parashaNameEnglish.replace(' ','%20')}).html"
with open(fileName, "w", encoding="utf-8") as file:
    file.write(htmlContent)

# wrap html header and footer and display a download button
htmlContentFull = f'{htmlStart}{htmlContent}{htmlFooter}'
downloadButton = f"""
<a download="{fileName}" href="data:text/html;charset=utf-8,{htmlContentFull.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;').replace('"', '&quot;').replace("'", '&#39;')}" target="_blank">
    <button>Download Differences as HTML</button>
</a>
"""
display(HTML(downloadButton))

### Levenshtein distance >15 between MT and SP for parasha Va’era (Exodus 6:2-9:35)

<h4><a href="https://www.stepbible.org/?q=version=NASB2020&reference=Exodus.6:9&options=HNVUG" target="_blank">Exodus 6:9</a></h4><p><b>SP:</b> וידבר מש<mark></mark>ה כן <mark></mark>ל<mark></mark>בני יש<mark></mark>ראל ולא ש<mark></mark>מעו אל מש<mark></mark>ה מקצר רוח ומעבדה קש<mark></mark>ה <mark>ויאמרו אל משה חדל נא ממנו ונעבדה את מצרים כי טוב לנו עבד את מצרים ממותנו במדבר </mark><br><b>MT:</b> וידבר מש<mark>ׁ</mark>ה כן <mark>א</mark>ל<mark> </mark>בני יש<mark>ׂ</mark>ראל ולא ש<mark>ׁ</mark>מעו אל מש<mark>ׁ</mark>ה מקצר רוח ומעבדה קש<mark>ׁ</mark>ה <mark></mark></p><p>Levenshtein Distance: 86</p>

<h4><a href="https://www.stepbible.org/?q=version=NASB2020&reference=Exodus.6:20&options=HNVUG" target="_blank">Exodus 6:20</a></h4><p><b>SP:</b> ויקח עמרם את יוכבד דדתו לו לאש<mark></mark>ה ותלד לו את אהרן ואת מש<mark></mark>ה ו<mark>את מרים אחותם ו</mark>ש<mark></mark>ני חיי עמרם ש<mark>ש</mark> וש<mark></mark>לש<mark></mark>ים ומאת ש<mark></mark>נה <br><b>MT:</b> ויקח עמרם את יוכבד דדתו לו לאש<mark>ׁ</mark>ה ותלד לו את אהרן ואת מש<mark>ׁ</mark>ה ו<mark></mark>ש<mark>ׁ</mark>ני חיי עמרם ש<mark>ׁבע</mark> וש<mark>ׁ</mark>לש<mark>ׁ</mark>ים ומאת ש<mark>ׁ</mark>נה </p><p>Levenshtein Distance: 23</p>

<h4><a href="https://www.stepbible.org/?q=version=NASB2020&reference=Exodus.7:9&options=HNVUG" target="_blank">Exodus 7:9</a></h4><p><b>SP:</b> כי ידבר אל<mark>י</mark>כם פרעה לאמר תנו לכם<mark> אות או</mark> מופת ואמרת אל אהרן קח את מטך והש<mark></mark>ל<mark>י</mark>ך לפני פרעה <mark>ו</mark>יהי לתנין <br><b>MT:</b> כי ידבר אל<mark></mark>כם פרעה לאמר תנו לכם<mark></mark> מופת ואמרת אל אהרן קח את מטך והש<mark>ׁ</mark>ל<mark></mark>ך לפני פרעה <mark></mark>יהי לתנין </p><p>Levenshtein Distance: 16</p>

<h4><a href="https://www.stepbible.org/?q=version=NASB2020&reference=Exodus.7:18&options=HNVUG" target="_blank">Exodus 7:18</a></h4><p><b>SP:</b> והדגה אש<mark>ר ביאר תמות ובאש</mark> היאר ונלאו מצרים לש<mark></mark>תות מים מן היאר <mark>וילך משה ואהרן אל פרעה ויאמרו אליו יהוה אלהי העברים שלחנו אליך לאמר שלח את עמי ויעבדני במדבר והנה לא שמעת עד כה כה אמר יהוה בזאת תדע כי אני יהוה הנה אנכי מכה במטה אשר בידי על המים אשר ביאר ונהפכו לדם והדגה אשר ביאר תמות ובאש היאר ונלאו מצרים לשתות מים מן היאר </mark><br><b>MT:</b> והדגה אש<mark>ׁ</mark>ר ביאר תמות ובאש<mark>ׁ</mark> היאר ונלאו מצרים לש<mark>ׁ</mark>תות מים מן היאר <mark></mark></p><p>Levenshtein Distance: 287</p>

...

<h4><a href="https://www.stepbible.org/?q=version=NASB2020&reference=Exodus.8:5&options=HNVUG" target="_blank">Exodus 8:5</a></h4><p><b>SP:</b> ויאמר מש<mark></mark>ה <mark>א</mark>ל<mark> </mark>פרעה התפאר עלי למתי<mark></mark> אעתיר לך ולעבדיך ולעמך להכרית הצפרדעים ממך ומבתיך <mark>ומעבדיך ומעמך </mark>רק ביאר תש<mark></mark>ארנה <br><b>MT:</b> ויאמר מש<mark>ׁ</mark>ה <mark></mark>ל<mark></mark>פרעה התפאר עלי למתי<mark>׀</mark> אעתיר לך ולעבדיך ולעמך להכרית הצפרדעים ממך ומבתיך <mark></mark>רק ביאר תש<mark>ׁ</mark>ארנה </p><p>Levenshtein Distance: 23</p>

<h4><a href="https://www.stepbible.org/?q=version=NASB2020&reference=Exodus.8:12&options=HNVUG" target="_blank">Exodus 8:12</a></h4><p><b>SP:</b> ויאמר יהוה אל מש<mark></mark>ה אמר אל אהרן נטה את <mark>ידך ב</mark>מטך והך את עפר הארץ ו<mark>י</mark>הי<mark></mark> לכנ<mark>י</mark>ם בכל ארץ מצרים <br><b>MT:</b> ויאמר יהוה אל מש<mark>ׁ</mark>ה אמר אל אהרן נטה את <mark></mark>מטך והך את עפר הארץ ו<mark>ה</mark>יה<mark></mark> לכנ<mark></mark>ם בכל ארץ מצרים </p><p>Levenshtein Distance: 16</p>

<h4><a href="https://www.stepbible.org/?q=version=NASB2020&reference=Exodus.8:19&options=HNVUG" target="_blank">Exodus 8:19</a></h4><p><b>SP:</b> וש<mark></mark>מתי פד<mark>ות בין עמי ובין עמך למחר יהיה האו</mark>ת הזה <mark>ויבא משה ואהרן אל פרעה ויאמרו אליו כה אמר יהוה שלח את עמי ויעבדני כי אם אינך משלח את עמי הנני משלח בך ובעבדיך ובעמך ובבתיך את הערב ומלאו בתי מצרים את הערב וגם האדמה אשר הם עליה והפליתי ביום ההוא את ארץ גשן אשר עמי עמד עליה לבלתי היות שם ערב למען תדע כי אני יהוה בקרב הארץ ושמתי פדות בין עמי ובין עמך למחר יהיה האות הזה </mark><br><b>MT:</b> וש<mark>ׂ</mark>מתי פד<mark></mark>ת בין עמי ובין עמך למחר יהיה הא<mark></mark>ת הזה <mark></mark></p><p>Levenshtein Distance: 347</p>

<h4><a href="https://www.stepbible.org/?q=version=NASB2020&reference=Exodus.9:5&options=HNVUG" target="_blank">Exodus 9:5</a></h4><p><b>SP:</b> ויש<mark>ם יהוה מועד לאמר מחר יעשה יהוה את</mark> הדבר הזה בארץ <mark>ויבא משה ואהרן אל פרעה ויאמרו אליו כה אמר יהוה אלהי העברים שלח את עמי ויעבדני כי אם מאן אתה לשלח ועודך מחזק בם הנה יד יהוה היה במקניך אשר בשדה בסוסים ובחמרים ובגמלים בבקר ובצאן דבר כבד מאד והפלא יהוה בין מקנה ישראל ובין מקנה מצרים ולא ימות מכל לבני ישראל דבר מחר יעשה יהוה את הדבר הזה בארץ </mark><br><b>MT:</b> ויש<mark>ׂ</mark>ם יהוה מועד לאמר מחר יעש<mark>ׂ</mark>ה יהוה<mark></mark> הדבר הזה בארץ <mark></mark></p><p>Levenshtein Distance: 319</p>

<h4><a href="https://www.stepbible.org/?q=version=NASB2020&reference=Exodus.9:19&options=HNVUG" target="_blank">Exodus 9:19</a></h4><p><b>SP:</b> ועתה ש<mark></mark>לח העז את מקנ<mark>יך ואת כל אשר לך בשדה כל האדם והבהמה אש</mark>ר ימצא בש<mark></mark>דה ולא יאסף הביתה וירד על<mark>יהם הברד ומתו ויבא משה ואהרן אל פרעה ויאמרו אליו כה אמר יהוה אלהי העברים שלח את עמי ויעבדני כי בפעם הזאת אני שלח את כל מגפתי על לבך ובעבדיך ובעמך בעבור תדע כי אין כמוני בכל הארץ כי עתה שלחתי את ידי ואכה אתך ואת עמך בדבר ותכחד מן הארץ ואולם בעבור זאת העמדתיך בעבור הראתיך את כחי ולמען ספר שמי בכל הארץ עודך מסתולל בעמי לבלתי שלחם הנני ממטיר כעת מחר ברד כבד מאד אשר לא היה כמהו במצרים למיום היסדה ועד עתה ועתה שלח העז את מקניך ואת כל אשר לך בשדה כל האדם והבהמה אשר ימצא בשדה ולא יאסף הביתה וירד עליהם הברד ומתו </mark><br><b>MT:</b> ועתה ש<mark>ׁ</mark>לח העז את מקנ<mark></mark>ך ואת כל אש<mark>ׁ</mark>ר לך בש<mark>ׂ</mark>דה כל האדם והבהמה אש<mark>ׁ</mark>ר ימצא בש<mark>ׂ</mark>דה ולא יאסף הביתה וירד על<mark></mark>הם הברד ומתו <mark></mark></p><p>Levenshtein Distance: 547</p>

# 5 - Comparison of spelling of proper nouns between SP and MT<a class="anchor" id="bullet5"></a>
##### [Back to ToC](#TOC)

This section focuses on comparing the spelling of proper nouns between the Samaritan Pentateuch (SP) and the Masoretic Text (MT). Proper nouns, including names of people, places, and unique terms, often exhibit variations in spelling

In [7]:
import collections

def collectProperNounSpellings(F, L, T, inputList):
    """
    Collect proper noun spellings and their associated word node numbers.
    Ensures only one tuple is stored for each lexeme-to-spelling mapping.
    """
    properNounsSpellings = {}
    for bookChapterVerse in inputList:
        verseNode = T.nodeFromSection(bookChapterVerse)
        wordNodes = L.d(verseNode, 'word')
        for wordNode in wordNodes:
            if F.sp.v(wordNode) == 'nmpr':  # Check if the word is a proper noun
                lex = F.lex.v(wordNode)    # Lexical form
                spelling = F.g_cons.v(wordNode)  # Spelling
                # Store only the first occurrence for each lex-to-cons mapping
                if lex not in properNounsSpellings or spelling not in {item[0] for item in properNounsSpellings[lex]}:
                    properNounsSpellings.setdefault(lex, []).append((spelling, wordNode))
    return properNounsSpellings
        
SPspellingDict = collectProperNounSpellings(Fsp, Lsp, Tsp, bookChapterVerseList) 
MTspellingDict = collectProperNounSpellings(Fmt, Lmt, Tmt, bookChapterVerseList)

In [8]:
from IPython.display import HTML, display

# Initialize HTML content
htmlContent = f'<h2>Spelling differences in proper nouns between SP and MT for parasha {parashaNameEnglish} ({parashaStart}-{parashaEnd})</h2>'

# Generate the HTML output
for lex, MTspellings in MTspellingDict.items():
    # Retrieve SP spellings, defaulting to an empty set if lex is not found
    SPspellings = SPspellingDict.get(lex, set())

    # Extract only the spellings (ignoring node numbers) for comparison
    MTspellingSet = {spelling for spelling, _ in MTspellings}
    SPspellingSet = {spelling for spelling, _ in SPspellings}

    # Compare the sets of spellings
    if MTspellingSet != SPspellingSet:
        # Print MT spelling with reference
        MTnode = list(MTspellings)[0][1]  # Get first tuple's node number
        book, chapter, verse = Tmt.sectionFromNode(MTnode)
        MTgloss = Fmt.gloss.v(MTnode)
        MTspelling = Fmt.g_cons_utf8.v(MTnode)

        # Build HTML output
        output = (
            f'<h4>Word: <b>{MTgloss}</b> '
            f'<a href="https://www.stepbible.org/?q=version=NASB2020&reference={book}.{chapter}:{verse}&options=HNVUG" target="_blank">'
            f'{book} {chapter}:{verse}</a></h4>'
            f'<ul><li><b>MT Spelling:</b> {MTspelling}</li>'
        )

        # Print SP spellings with reference
        if SPspellings:
            SPnode = list(SPspellings)[0][1]  # Get first tuple's node number
            SPspelling = Fsp.g_cons_utf8.v(SPnode)
            output += f'<li><b>SP Spelling:</b> {SPspelling}</li></ul>'
        else:
            output += '<li><b>SP Spelling:</b> None</li></ul>'

        # Append the output to the HTML content
        htmlContent += output

# Save the HTML content to a file
fileName = f"spelling_differences_SP_MT({parashaNameEnglish.replace(' ','%20')}).html"
with open(fileName, "w", encoding="utf-8") as file:
    file.write(htmlContent)

# Display the HTML content in the notebook
display(HTML(htmlContent))

# wrap html header and footer and display a download button
htmlContentFull = f'{htmlStart}{htmlContent}{htmlFooter}'
downloadButton = f"""
<a download="{fileName}" href="data:text/html;charset=utf-8,{htmlContentFull.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;').replace('"', '&quot;').replace("'", '&#39;')}" target="_blank">
    <button>Download Differences as HTML</button>
</a>
"""
display(HTML(downloadButton))

# 6 - References and acknowledgement <a class="anchor" id="bullet6"></a>
##### [Back to ToC](#TOC)

<a class="anchor" id="bullet1"><sup>1</sup></a> Christian Canu Højgaard, Martijn Naaijer, & Stefan Schorch. (2023). Text-Fabric Dataset of the Samaritan Pentateuch. Zenodo. https://doi.org/10.5281/zenodo.7734632

<a class="anchor" id="bullet2"><sup>2</sup> [Notebook created by Martijn Naaijer](https://github.com/DT-UCPH/sp/blob/main/notebooks/combine_sp_with_mt_data.ipynb)

# 7 - Required libraries <a class="anchor" id="bullet7"></a>
##### [Back to ToC](#TOC)

The scripts in this notebook require (beside `text-fabric`) the following Python libraries to be installed in the environment:

    collections
    difflib
    Levenshtein

You can install any missing library from within Jupyter Notebook using either`pip` or `pip3`.

# 8 - Notebook version details<a class="anchor" id="bullet8"></a>
##### [Back to ToC](#TOC)

<div style="float: left;">
  <table>
    <tr>
      <td><strong>Author</strong></td>
      <td>Tony Jurg</td>
    </tr>
    <tr>
      <td><strong>Version</strong></td>
      <td>1.1</td>
    </tr>
    <tr>
      <td><strong>Date</strong></td>
      <td>5 March 2025</td>
    </tr>
  </table>
</div>